### Combinations
Order doesn't matter. (1, 2) == (2, 1)

Note: to avoid duplicates, first sort the array then use the set to skip over "used" elements in the for loop. See Permutations section.

In [63]:
from itertools import combinations

list(combinations([1, 2, 5, 7], 3))

[(1, 2, 5), (1, 2, 7), (1, 5, 7), (2, 5, 7)]

In [85]:
def combinations(arr: list[int], r: int) -> list[list[int]]:
    result = []
    arr.sort()

    def backtrack(start: int, path: list[int]) -> None:
        print(path)
        if len(path) == r:
            result.append(path.copy())
            return  # remove this to get all possible lengths
        
        used = set()
        for i in range(start, len(arr)):
            if arr[i] in used:
                continue
            used.add(arr[i])
            path.append(arr[i])
            backtrack(i + 1, path)  # remove “+ 1” to allow repeats.
            path.pop()
    
    backtrack(0, [])
    return result

###
combinations([1, 2, 3, 1], 3)

[]
[1]
[1, 1]
[1, 1, 2]
[1, 1, 3]
[1, 2]
[1, 2, 3]
[1, 3]
[2]
[2, 3]
[3]


[[1, 1, 2], [1, 1, 3], [1, 2, 3]]

In [ ]:
# if repeats are allowed, use
from itertools import combinations_with_replacement

### Subsets
Combos of all lengths

In [ ]:
# Recursive
def subsets_recursive(items: list) -> list[list]:
    if not items:
        return [[]]
    # all except last element
    subsets = subsets_recursive(items[:-1])
    return subsets + [s + [items[-1]] for s in subsets]


# Iterative
def subsets_iterative(items: list) -> list[list]:
    subsets = [[]]
    for item in items:
        # add this item to everything in subsets
        for i in range(len(subsets)):
            subsets.append(subsets[i].copy())
            subsets[-1].append(item)
    return subsets


# Binary counting
def subsets_binary(items):
    n = len(items)
    all_subsets = []
    
    # Generate all binary numbers from 0 to 2^n - 1
    for i in range(1 << n):  # 1 << n is 2^n
        # 1 << j creates a binary number with only the jth bit set to 1
        # i & (1 << j) performs a bitwise AND operation
        # This determines whether to include the jth element in the current subset
        subset = [items[j] for j in range(n) if (i & (1 << j))]
        all_subsets.append(subset)
    
    return all_subsets

subsets_binary([1, 2, 3])

[[], [1], [2], [1, 2], [3], [1, 3], [2, 3], [1, 2, 3]]

For de-duped cases, need to avoid using the same item twice on the same tree level.

In [ ]:
# stack/queue based solution (doesn't matter if stack of queue)
def subsets_iterative_deduped_stack(items):
    items.sort()
    result = []

    stack = [([], 0)]
    while stack:
        subset, start_idx = stack.pop()
        result.append(subset)
        for i in range(start_idx, len(items)):
            # dedupe here
            if i > start_idx and items[i] == items[i - 1]:
                continue  # don't repeat it
            stack.append((subset + [items[i]], i + 1))

    return result

# solution without sorting (keep a set per level of tree)
def subsets_iterative_deduped_no_sorting(items):
    result = []
    subsets_this_level = [([], 0)]

    while subsets_this_level:
        used = set()
        subsets_next_level = []
        for subset, start_idx in subsets_this_level:
            result.append(subset)
            for i in range(start_idx, len(items)):
                # dedupe here
                if items[i] in used:
                    continue
                used.add(items[i])
                subsets_next_level.append((subset + [items[i]], i + 1))
        
        subsets_this_level = subsets_next_level
    return result


def subsets_recursive_deduped(items):
    # Sort the items to group duplicates together
    items.sort()
    result = []
    
    def backtrack(start, current_subset):
        # Add the current subset to the result
        result.append(current_subset[:])
        
        # Try adding each remaining item
        for i in range(start, len(items)):
            # Skip duplicates - only take first occurrence at each level
            if i > start and items[i] == items[i - 1]:
                continue
                
            # Include this element
            current_subset.append(items[i])
            # Recurse to the next position
            backtrack(i + 1, current_subset)
            # Backtrack
            current_subset.pop()
    
    backtrack(0, [])
    return result

### Permutations
Order matters. (1, 2) != (2, 1)

In [10]:
from itertools import permutations

list(permutations([1, 2, 3], 2))

[(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2)]

In [72]:
def permutations(arr: list[int], r: int, allow_repeats: bool = True):
    if r == 0:
        return [[]]

    result = []
    used = set() if not allow_repeats else None  # create only if needed
    for i in range(len(arr)):
        # Use the current item as the first element
        current = arr[i]
        if not allow_repeats:
            if current in used:
                # skip to avoid repeats
                continue
            used.add(current)
        # Get the remaining items
        remaining = arr[:i] + arr[i + 1:]
        
        # Generate all permutations of remaining items
        for p in permutations(remaining, r - 1, allow_repeats):
            result.append([current] + p)
    
    return result

print(permutations([1, 1, 2], 3, allow_repeats=False))
print(permutations([1, 1, 2], 3, allow_repeats=True))

[[1, 1, 2], [1, 2, 1], [2, 1, 1]]
[[1, 1, 2], [1, 2, 1], [1, 1, 2], [1, 2, 1], [2, 1, 1], [2, 1, 1]]
